In [2]:
import os
import pandas as pd

def load_result_csv_files(directory):
    result_dfs = []
    file_paths = []
    for root, _, files in os.walk(directory):
        for filename in files:
            if filename.endswith("results.csv"):
                file_path = os.path.join(root, filename)
                relative_path = os.path.relpath(file_path, directory)
                df = pd.read_csv(file_path)
                result_dfs.append(df)
                file_paths.append(relative_path)
    return result_dfs, file_paths

directory_path = "/workdir/optimal-summaries-public/_models2"
result_dfs, file_paths = load_result_csv_files(directory_path)
len(result_dfs)

13

In [47]:
pd.set_option('display.max_rows', None)

In [48]:
combined_df = pd.concat(result_dfs, ignore_index=True)
combined_df.drop(columns=['Seed']).groupby(["Dataset", "Pruning", "Model", "Finetuned", "Split"]).agg(["mean", "std"])

AUC  \
                                                                                      mean   
Dataset              Pruning         Model                       Finetuned Split             
mimic                Before          atomics_sum2atomics_True    False     test   0.914252   
                                                                           val    0.914574   
                                     original                    False     test   0.915310   
                                                                           val    0.917164   
                                     shared_encode_time_dim_True False     test   0.916582   
                                                                           val    0.919516   
                     Greedy          atomics_sum2atomics_True    False     test   0.903354   
                                                                           val    0.903581   
                                                                 True      test   0.896346   
                                                                           val    0.897360   
                                     shared_encode_time_dim_True False     test   0.916046   
                                                                           val    0.918871   
                                                                 True      test   0.915064   
                                                                           val    0.917465   
                     importance      original                    True      test   0.916842   
                                                                           val    0.919382   
                     sparse_learning atomics_sum2atomics_True    True      test   0.915010   
                                                                           val    0.916506   
                                     original                    True      test   0.916475   
                                                                           val    0.919034   
                                     shared_encode_time_dim_True True      test   0.916582   
                                                                           val    0.919516   
spoken_arabic_digits Before          atomics_sum2atomics_True    False     test   0.993706   
                                                                           val    0.995062   
                                     original                    False     test   0.993961   
                                                                           val    0.994266   
                                     shared_encode_time_dim_True False     test   0.998937   
                                                                           val    0.999135   
                     Greedy          atomics_sum2atomics_True    False     test   0.603468   
                                                                           val    0.604591   
                                                                 True      test   0.661223   
                                                                           val    0.661726   
                                     shared_encode_time_dim_True False     test   0.994029   
                                                                           val    0.995194   
                                                                 True      test   0.998626   
                                                                           val    0.998885   
                     importance      original                    True      test   0.994188   
                                                                           val    0.994717   
                     sparse_learning atomics_sum2atomics_True    True      test   0.994477   
                                                                           val    0.994894   
                                     original

In [49]:
combined_df.drop(columns=["Seed", "Dataset"]).groupby(["Pruning", "Model", "Finetuned", "Split"]).agg(["mean", "std"])

AUC  \
                                                                 mean   
Pruning         Model                       Finetuned Split             
Before          atomics_sum2atomics_True    False     test   0.956453   
                                                      val    0.956950   
                original                    False     test   0.954636   
                                                      val    0.955715   
                shared_encode_time_dim_True False     test   0.960661   
                                                      val    0.961497   
Greedy          atomics_sum2atomics_True    False     test   0.801072   
                                                      val    0.801997   
                                            True      test   0.835347   
                                                      val    0.836052   
                shared_encode_time_dim_True False     test   0.953566   
                                                      val    0.954566   
                                            True      test   0.960712   
                                                      val    0.961350   
importance      original                    True      test   0.955515   
                                                      val    0.957049   
sparse_learning atomics_sum2atomics_True    True      test   0.957063   
                                                      val    0.957542   
                original                    True      test   0.958334   
                                                      val    0.958964   
                shared_encode_time_dim_True True      test   0.960661   
                                                      val    0.961497   

                                                                       \
                                                                  std   
Pruning         Model                       Finetuned Split             
Before          atomics_sum2atomics_True    False     test   0.034669   
                                                      val    0.035116   
                original                    False     test   0.043094   
                                                      val    0.042300   
                shared_encode_time_dim_True False     test   0.035943   
                                                      val    0.034702   
Greedy          atomics_sum2atomics_True    False     test   0.174242   
                                                      val    0.174623   
                                            True      test   0.193065   
                                                      val    0.193509   
                shared_encode_time_dim_True False     test   0.034166   
                                                      val    0.033711   
                                            True      test   0.036664   
                                                      val    0.035741   
importance      original                    True      test   0.042381   
                                                      val    0.041390   
sparse_learning atomics_sum2atomics_True    True      test   0.034606   
                                                      val    0.034124   
                original                    True      test   0.033933   
                                                      val    0.032871   
                shared_encode_time_dim_True True      test   0.035943   
                                                      val    0.034702   

                                                                  ACC  \
                                                                 mean   
Pruning         Model                       Finetuned Split             
Before          atomics_sum2atomics_True    False     test   0.831119   
                                                      val    0.832722   
                original                    False